In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
def count_clusters(table):
    
    counts = table["cls.cluster"].value_counts(sort=False, ascending=True)
    maxval = table["cls.cluster"].max()
    count_dict = {}

    for i in range(0, maxval):
        count_dict[i] = 0

    for i in counts.keys():
        count_dict[i] = counts[i]
    
    # dict key 0 holds total value
    count_dict[0] = counts.sum()
    
    return count_dict



def compare_results(region, group):
    edgeRdir = "G:\\My Drive\\biostat\\02_mlcompbio\\project\\results_topTags\\"
    mbclusterdir = "G:\\My Drive\\biostat\\02_mlcompbio\\project\\results_MBCDeg\\"

    edgeR = pd.read_csv("{0}tags{1}_{2}.csv".format(edgeRdir, region, group))
    mbcluster = pd.read_csv("{0}{1}_ordered_{2}.csv".format(mbclusterdir, region, group))

    edgeR = edgeR[edgeR["PValue"] < 0.001]

    joined = mbcluster.merge(edgeR, how="left", right_on="Unnamed: 0", left_on="mydata.GeneID")
    joined.dropna(inplace=True)
    #joined.to_csv("{0}combined_{1}_{2}.csv".format(region, group))

    return count_clusters(joined), count_clusters(mbcluster)




In [3]:
outputdir = "G:\\My Drive\\biostat\\02_mlcompbio\\project\\results_output\\"
file = outputdir + "\\clusters.txt"

regions = ["FWM", "HIP", "PCx", "TCx"] # values: ["FWM", "HIP", "PCx", "TCx"]
groups = ["AD", "FULL"] # values: ["AD", "FULL"]

with open(file, "a") as writer:
    for region in regions:
        for group in groups:
            counts_all, counts_mbclust = compare_results(region, group)
            writer.write(region + ", " + group + "\n")
            print(counts_all, file=writer)
            print(counts_mbclust, file=writer)
            writer.write("\n")

In [21]:
def find_common():
    regions = ["FWM", "HIP", "PCx", "TCx"] # values: ["FWM", "HIP", "PCx", "TCx"]
    groups = ["FULL"] # values: ["AD", "FULL"]
    count = False
    edgeRdir = "G:\\My Drive\\biostat\\02_mlcompbio\\project\\results_topTags\\"
    for region in regions:
        for group in groups:
            if count is False:
                edgeR = pd.read_csv("{0}tags{1}_{2}.csv".format(edgeRdir, region, group))
                edgeR = edgeR[edgeR["PValue"] < 0.001]
                print(edgeR.shape)
                count = True
            else:
                edgeRcomp = pd.read_csv("{0}tags{1}_{2}.csv".format(edgeRdir, region, group))
                edgeRcomp = edgeRcomp[edgeRcomp["PValue"] < 0.001]
                edgeR = edgeR.merge(edgeRcomp, how="left", right_on="Unnamed: 0", left_on="Unnamed: 0",)
                print(edgeR.shape)
    return edgeR


find_common().dropna()["Unnamed: 0"]

(3931, 6)
(3931, 11)
(3931, 16)
(3931, 21)


C:\Users\Vasilis\AppData\Local\Temp\ipykernel_15580\1369454948.py:16: FutureWarning: Passing 'suffixes' which cause duplicate columns {'PValue_x', 'logCPM_x', 'FDR_x', 'logFC_x', 'F_x'} in the result is deprecated and will raise a MergeError in a future version.
  edgeR = edgeR.merge(edgeRcomp, how="left", right_on="Unnamed: 0", left_on="Unnamed: 0",)


67      LOC124902040
270             RARG
394          COL27A1
408           ZRANB2
496             EVC2
631             HIC1
834           MINDY3
922      FPGT-TNNI3K
1600          TNNI3K
2045             SMO
2912           MTSS2
3125           ITGB5
3464           TEAD2
3482         SLC6A12
Name: Unnamed: 0, dtype: object

In [52]:
from bioinfokit import analys, visuz
import os 

os.chdir(outputdir)

mbcluster_heat = mbcluster.iloc[0:100,].sort_values(by=["cls.cluster"])

# load dataset as pandas dataframe
if group == "FULL":
    heat = mbcluster_heat[["X1.1", "X2.1", "X3.1", "X4.1"]]
elif group == "AD":
    heat = mbcluster_heat[["X1.1", "X2.1"]]
heat = heat.set_index(mbcluster_heat["mydata.GeneID"])

visuz.gene_exp.hmap(df=heat, rowclus=False, colclus=False, dim=(6, 12), tickfont=(0, 0))